# Wartungskosten Maximus

In [1]:
import shutil

import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

images_path = Path('images_wartung')
if images_path.exists():
    shutil.rmtree(images_path)
images_path.mkdir(parents=True, exist_ok=True)

In [2]:
tanken_df = pd.read_csv('Maximus.csv',
                         skiprows=6,
                         skipfooter=144,
                         engine='python',
                         index_col=1,
                         usecols=[0, 2, 6],
                         encoding='utf-8',
                         header=0,
                         names=['Tachostand (km)', 'Datum', 'Kosten'])
tanken_df.index = pd.to_datetime(tanken_df.index)
tanken_df['Subtyp'] = 'Tanken'
tanken_df.head()

,Tachostand (km),Kosten,Subtyp
Datum,,,
2015-04-01 18:59:00,76157,60.18,Tanken
2015-04-04 19:24:00,77118,71.20,Tanken
2015-04-13 18:02:00,78048,71.83,Tanken
2015-04-24 17:42:00,78980,72.46,Tanken
2015-04-30 11:00:00,79833,63.05,Tanken


# Import Unterhalt

In [3]:
unterhalt_df = pd.read_csv('Maximus.csv',
                           skiprows=232,
                           skipfooter=37,
                           engine='python',
                           usecols=[0, 1, 2, 3, 7],
                           index_col=1,
                           encoding='utf-8')
unterhalt_df.index = pd.to_datetime(unterhalt_df.index)
unterhalt_df.head()

,Beschreibung,Tachostand (km),Kosten,Subtyp
Datum,,,,
2015-02-22 00:00:00,Kredit,76020.0,-17000.0,Finanzierung
2015-02-23 00:00:00,Kaufbetrag,76020.0,16180.0,Finanzierung
2015-03-02 00:00:00,Kreditrate 0,76020.0,350.0,Finanzierung
2015-03-09 14:00:00,Anzahlung,76020.0,500.0,Finanzierung
2015-03-30 07:19:00,Garantie,76020.0,499.0,Finanzierung


## Agglomerierter Unterhalt

In [4]:
joined_df = pd.concat([unterhalt_df, tanken_df])
joined_df = joined_df.sort_index()
joined_df['Weg'] = joined_df['Tachostand (km)'].diff()
joined_df.loc[joined_df.Weg.isna(), ['Weg']] = 0
#joined_df.loc[(joined_df.Subtyp == 'Wartung') |
#              (joined_df.Subtyp == 'Verschleißteile') |
#              (joined_df.Subtyp == 'Reparatur') |
#              (joined_df.Subtyp == 'Pflege') |
#              (joined_df.Subtyp == 'Unfallschaden') |
#              (joined_df.Subtyp == 'Tuning'), ['Subtyp']] = 'Wartung'
joined_df = joined_df.reset_index()
joined_df

,Datum,Beschreibung,Tachostand (km),Kosten,Subtyp,Weg
0,2015-02-22 00:00:00,Kredit,76020.0,-17000.00,Finanzierung,0.0
1,2015-02-23 00:00:00,Kaufbetrag,76020.0,16180.00,Finanzierung,0.0
2,2015-03-02 00:00:00,Kreditrate 0,76020.0,350.00,Finanzierung,0.0
3,2015-03-09 14:00:00,Anzahlung,76020.0,500.00,Finanzierung,0.0
4,2015-03-30 07:19:00,Garantie,76020.0,499.00,Finanzierung,0.0
...,...,...,...,...,...,...
321,2021-11-16 08:08:00,NaN,252260.0,83.17,Tanken,260.0
322,2021-12-04 17:44:00,NaN,253194.0,92.06,Tanken,934.0
323,2021-12-18 11:44:00,Diagnose Steuerkette,253640.0,53.79,Reparatur,446.0
324,2021-12-21 00:00:00,Abmeldung,253640.0,7.80,Steuer,0.0


In [5]:
subtypes = joined_df.Subtyp.unique()
subtypes

array(['Finanzierung', 'Wartung', 'Verschleißteile', 'Tanken', 'Steuer',
       'Versicherung', 'Reparatur', 'Pflege', 'Unfallschaden', 'Tuning'],
      dtype=object)

In [6]:
freq='1Y'
aggregated_df = pd.DataFrame(index=pd.date_range('2015-01-01', '2022-12-31', freq=freq))

for ii in range(len(subtypes)):
    aggregated_df.loc[aggregated_df.index[0],
                      subtypes[ii]] = joined_df.query('Datum <= "%s" & Subtyp == "%s"' % (aggregated_df.index[0], 
                                                                                          subtypes[ii])).Kosten.sum()

for idx in aggregated_df.index[1:]:
    for ii in range(len(subtypes)):
        aggregated_df.loc[idx,
                          subtypes[ii]] = joined_df.query('Datum <= "%s" & Subtyp == "%s"' % (idx, 
                                                                                              subtypes[ii])).Kosten.sum()
        
        
aggregated_df.iloc[1:, :] = aggregated_df.diff().iloc[1:, :]
#aggregated_df.loc[aggregated_df.Finanzierung < 0, ['Finanzierung']] = 0
aggregated_df.head()


,Finanzierung,Wartung,Verschleißteile,Tanken,Steuer,Versicherung,Reparatur,Pflege,Unfallschaden,Tuning
2015-12-31,4149.00,487.99,886.60,2201.40,230.0,738.69,336.29,0.0,0.0,0.0
2016-12-31,4910.00,1174.68,504.75,2376.20,338.8,707.38,0.00,0.0,0.0,0.0
2017-12-31,8143.36,393.89,0.00,2707.18,230.0,780.66,74.59,12.0,1200.0,0.0
2018-12-31,0.00,642.03,0.00,2240.60,230.0,768.32,1003.85,0.0,0.0,0.0
2019-12-31,0.00,24.89,895.00,1515.74,230.0,732.02,300.00,7.9,0.0,1296.0


In [7]:
aggregated_df.Wartung

2015-12-31     487.99
2016-12-31    1174.68
2017-12-31     393.89
2018-12-31     642.03
2019-12-31      24.89
2020-12-31     640.47
2021-12-31      29.75
2022-12-31       0.00
Freq: A-DEC, Name: Wartung, dtype: float64

In [8]:
sum(aggregated_df.Wartung) / len(aggregated_df.Wartung) / 12

35.35104166666667

In [9]:
wartung_df = joined_df[joined_df.Subtyp == 'Wartung'].set_index('Datum').drop(['Weg', 'Subtyp'], axis=1)
wartung_df.to_excel('wartung_bmw.xlsx')
wartung_df

,Beschreibung,Tachostand (km),Kosten
Datum,,,
2015-03-30 15:16:00,Kundendienst,76020.0,399.99
2015-11-04 16:20:00,Radwechsel,101599.0,88.00
2016-01-12 09:55:00,Ölwechsel,107211.0,301.37
2016-04-27 07:55:00,Reifenwechseln,118740.0,80.00
2016-07-21 00:00:00,Technische Durchsicht,127227.0,253.41
2016-11-22 00:00:00,Öl- und Reifenwechsel,138875.0,539.90
2017-03-24 00:00:00,Reifenwechseln,147744.0,24.89
2017-04-25 17:49:00,Bremsflüssigkeit wechseln,150551.0,86.48
2017-10-13 00:00:00,Ölwechsel,170748.0,282.52
